# Jupyter Notebook for mainnet deployment


In [ ]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))


# Set Up Environment

In [ ]:
from iconservice import AddressPrefix, Address
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder,DepositTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass
from dotenv import dotenv_values

response = {}
print(1)

In [ ]:
config = dotenv_values("./_deployment/.env.jupyter")

EXA = 10 ** 18
ICX = 10 ** 18

NETWORK = config["NETWORK"]

PREPS_LIST_FILE = config["PREPS_LIST_FILE"]
with open(PREPS_LIST_FILE, "r") as file:
    PREP_LIST = json.load(file)

# PREP_LIST = config['PREPS_LIST'].split(",")
TIMESTAMP = int(config["TIMESTAMP"])

PRIVATE_KEY = config["PRIVATE_KEY"]

GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"

#lending and borrow reward
LENDING_BORROW_PERCENTAGE = int(config["LENDING_BORROW_PERCENTAGE"]) * EXA // 100
#LP and OMM staking reward
LP_OMM_STAKING_PERCENTAGE = int(config["LP_OMM_STAKING_PERCENTAGE"]) * EXA // 100
#DAO reward percentage
DAO_DIST_PERCENTAGE = int(config["DAO_DIST_PERCENTAGE"]) * EXA // 100
#Worker reward 
WORKER_DIST_PERCENTAGE = int(config["WORKER_DIST_PERCENTAGE"]) * EXA // 100

assert (LENDING_BORROW_PERCENTAGE + LP_OMM_STAKING_PERCENTAGE + DAO_DIST_PERCENTAGE + WORKER_DIST_PERCENTAGE) // EXA == 1

ICX_EMISSION = int(config["ICX_PERCENTAGE"])
OICX_EMISSION = int(config["OICX_EMISSION"]) * ICX_EMISSION * EXA // 10000
DICX_EMISSION = int(config["DICX_EMISSION"]) * ICX_EMISSION * EXA // 10000

USDS_EMISSION = int(config["USDS_PERCENTAGE"])
OUSDS_EMISSION = int(config["OUSDS_EMISSION"]) * USDS_EMISSION * EXA // 10000
DUSDS_EMISSION = int(config["DUSDS_EMISSION"]) * USDS_EMISSION * EXA // 10000

IUSDC_EMISSION = int(config["IUSDC_PERCENTAGE"])
OIUSDC_EMISSION = int(config["OIUSDC_EMISSION"]) * IUSDC_EMISSION * EXA // 10000
DIUSDC_EMISSION = int(config["DIUSDC_EMISSION"]) * IUSDC_EMISSION * EXA // 10000

assert (OICX_EMISSION + DICX_EMISSION + OUSDS_EMISSION + DUSDS_EMISSION + OIUSDC_EMISSION + DIUSDC_EMISSION) // EXA == 1

#LP and OMM staking reward

OMM_SICX_DIST_PERCENTAGE = int(config["OMM_SICX_DIST_PERCENTAGE"]) * EXA // 100
OMM_USDS_DIST_PERCENTAGE = int(config["OMM_USDS_DIST_PERCENTAGE"]) * EXA // 100
OMM_USDC_DIST_PERCENTAGE = int(config["OMM_USDC_DIST_PERCENTAGE"]) * EXA // 100
OMM_DIST_PERCENTAGE = int(config["OMM_DIST_PERCENTAGE"]) * EXA // 100

assert (OMM_SICX_DIST_PERCENTAGE + OMM_USDS_DIST_PERCENTAGE + OMM_USDC_DIST_PERCENTAGE + OMM_DIST_PERCENTAGE) // EXA == 1

DEPOSIT_ICX_AMOUNT = int(config["DEPOSIT_ICX_AMOUNT"])
FEE_SHARING_TX_LIMIT = int(config["FEE_SHARING_TX_LIMIT"])
LOAN_ORIGINATION_PERCENTAGE = int(float(config["LOAN_ORIGINATION_PERCENTAGE"]) * EXA)
MINIMUM_OMM_STAKE = int(config["MINIMUM_OMM_STAKE"]) * EXA
UNSTAKING_PERIOD = int(config["UNSTAKING_PERIOD"])

# id from dex
OMM_SICX_ID = int(config["OMM_SICX_ID"])
OMM_USDS_ID = int(config["OMM_USDS_ID"])
OMM_USDC_ID = int(config["OMM_USDC_ID"])

DEPLOYER_WALLERT_KEYSTORE = int(config["DEPLOYER_WALLERT_KEYSTORE"])
DEPLOYER_WALLERT_KEYSTORE_PASSWORD = int(config["DEPLOYER_WALLERT_KEYSTORE_PASSWORD"])

_contract_file_loaded = False
print(1)

In [ ]:
connections = {
    "mainnet": {"iconservice": "https://ctz.solidwallet.io", "nid": 1},
    "yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
    "euljiro": {"iconservice": "https://test-ctz.solidwallet.io", "nid": 2},
    "pagoda": {"iconservice": "https://zicon.net.solidwallet.io", "nid": 80},
    "custom": {"iconservice": "http://18.237.205.52:9000/", "nid": 3},
    "local": {"iconservice": "http://localhost:9000/", "nid": 3},
    "sejong": {"iconservice": "https://sejong.net.solidwallet.io", "nid": 83}
}

env = connections[NETWORK]

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]


@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result


print(f"newtwork -> {NETWORK}")
print(f"nid -> {NID}")

### Import deployer wallet

In [3]:
keystore_password = input(f'Password for deployer wallet keystore? ')
btest_wallet = KeyWallet.load(DEPLOYER_WALLERT_KEYSTORE, keystore_password)
print(deployer_wallet.get_address())
print(icon_service.get_balance(deployer_wallet.get_address()))

Password for deployer wallet keystore? s@@nu2131


NameError: name 'KeyWallet' is not defined

In [ ]:
# Run this in case you want to import previous contract addresses
contracts_file_custom = config["CONTRACT_ADDRESS_FILE"]
with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)
    _contract_file_loaded = True

## Helper Contracts and Variables

In [ ]:
if not _contract_file_loaded:
    contracts = {}
    contracts['sicx'] = config["SICX"]
    contracts['staking'] = config["STAKING"]
    contracts['iusdc'] = config["IUSDC"]
    contracts["usds"] = config.get("USDS")
    contracts['bandOracle'] = config["BAND_ORACLE"]
    contracts["dex"] = config["DEX"]

## Deploying all contracts

In [ ]:
def deploy_contract(item):
    params = {}
    if item == "workerToken":
        params = {'_initialSupply': int(config.get("worker_token_initial")), '_decimals': 18}
    if item == "rewardDistribution":
        DISTRIBUTION_PERCENTAGE = [
            {"recipient": "worker", "percentage": f'{WORKER_DIST_PERCENTAGE}'},
            {"recipient": "daoFund", "percentage": f'{DAO_DIST_PERCENTAGE}'},
            {"recipient": "lendingBorrow", "percentage": f'{LENDING_BORROW_PERCENTAGE}'},
            {"recipient": "liquidityProvider", "percentage": f'{LP_OMM_STAKING_PERCENTAGE}'}
        ]
        params = {"_distPercentage": DISTRIBUTION_PERCENTAGE, '_startTimestamp': TIMESTAMP}

    if item not in ['addressProvider',"workerToken","feeProvider","daoFund"]:
        params['_addressProvider'] = contracts['addressProvider']

    deploy_transaction = DeployTransactionBuilder() \
        .from_(deployer_wallet.get_address()) \
        .to(GOVERNANCE_ADDRESS) \
        .nid(NID) \
        .nonce(100) \
        .content_type("application/zip") \
        .content(gen_deploy_data_content(item)) \
        .params(params) \
        .build()


    step_limit = icon_service.estimate_step(deploy_transaction) + 100000

    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    message = res.get('failure')
    status = res['status']
    print(tx_hash)
    print(status)
    print(message)
    response[f"deploy_contract_{item}"] = res['status']
    return res.get('scoreAddress', '')

In [ ]:
#deploy addresss provider contract
contracts['addressProvider'] = deploy_contract('addressProvider')

In [ ]:
assert response.get('deploy_contract_addressProvider',0) == 1,"addressProvider is not deployed." 

In [ ]:
#deploy rest of contracts
deploy = ['daoFund', 'delegation', 'feeProvider', 'governance',
          'lendingPool', 'lendingPoolCore', 'lendingPoolDataProvider',
          'liquidationManager', 'ommToken', 'priceOracle', 'rewardDistribution',
          'workerToken', 'stakedLp']

directory = os.getcwd()
for item in deploy:
    print(f"\n Deploying {item} contract\n")
    contracts[item] = deploy_contract(item)

## Retry contract deploy

In [ ]:
for item in deploy:
    if item not in contracts.keys() or contracts[item] == "" or contracts[item] == None:
        contracts[item] = deploy_contract(item)

## reading configuration for otoken and dtoken

In [ ]:
def _deploy_token_contract(token, params):
    name = params["name"]
    symbol = params["symbol"]
    decimals = params["decimals"]
    print(f"deploying {name}::{symbol}::{decimals}")
    params = {"_name": name, "_symbol": symbol, "_decimals": decimals, '_addressProvider': contracts['addressProvider']}
    deploy_transaction = DeployTransactionBuilder() \
        .from_(deployer_wallet.get_address()) \
        .to(GOVERNANCE_ADDRESS) \
        .nid(NID) \
        .nonce(100) \
        .content_type("application/zip") \
        .content(gen_deploy_data_content(token)) \
        .params(params) \
        .build()


    step_limit = icon_service.estimate_step(deploy_transaction) + 100000

    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    response[f"deploy_{token}_contract_{symbol}"] = res['status']
    print(res)
    return res.get('scoreAddress', '')

In [ ]:
TOKEN_CONFIGURATION_FILE = config["TOKEN_CONFIGURATION_FILE"]
with open(TOKEN_CONFIGURATION_FILE, "r") as file:
    data = json.load(file)

for params in data['dToken']:
    contracts[params['symbol']] = _deploy_token_contract('dToken', params)

for params in data['oToken']:
    contracts[params['symbol']] = _deploy_token_contract('oToken', params)

In [ ]:
assert response.get('deploy_contract_addressProvider',0) == 1,"addressProvider is not deployed."
assert response.get('deploy_contract_daoFund',0) == 1,"daoFund is not deployed."
assert response.get('deploy_contract_delegation',0) == 1,"delegation is not deployed."
assert response.get('deploy_contract_feeProvider',0) == 1,"feeProvider is not deployed."
assert response.get('deploy_contract_governance',0) == 1,"governance is not deployed."
assert response.get('deploy_contract_lendingPool',0) == 1,"lendingPool is not deployed."
assert response.get('deploy_contract_lendingPoolCore',0) == 1,"lendingPoolCore is not deployed."
assert response.get('deploy_contract_lendingPoolDataProvider',0) == 1,"lendingPoolDataProvider is not deployed."
assert response.get('deploy_contract_liquidationManager',0) == 1,"liquidationManager is not deployed."
assert response.get('deploy_contract_ommToken',0) == 1,"ommToken is not deployed."
assert response.get('deploy_contract_priceOracle',0) == 1,"priceOracle is not deployed."
assert response.get('deploy_contract_rewardDistribution',0) == 1,"rewardDistribution is not deployed."
assert response.get('deploy_contract_workerToken',0) == 1,"workerToken is not deployed."
assert response.get('deploy_contract_stakedLp',0) == 1,"stakedLp is not deployed."
assert response.get('deploy_dToken_contract_dUSDS',0) == 1,"dUSDS is not deployed."
assert response.get('deploy_dToken_contract_dICX',0) == 1,"dICX is not deployed."
assert response.get('deploy_dToken_contract_dIUSDC',0) == 1,"dIUSDC is not deployed."
assert response.get('deploy_oToken_contract_oUSDS',0) == 1,"oUSDS is not deployed."
assert response.get('deploy_oToken_contract_oICX',0) == 1,"oICX is not deployed."
assert response.get('deploy_oToken_contract_oIUSDC',0) == 1,"oIUSDC is not deployed."
print("----All SCORE deployed successfully----")

In [ ]:
#create dummy contract address for liquidity pools
contracts['OMM/sICX'] = Address.from_data(AddressPrefix.CONTRACT,
                                          f"{contracts['ommToken']}/{contracts['sicx']}".encode("UTF-8"))
contracts['OMM/USDS'] = Address.from_data(AddressPrefix.CONTRACT,
                                          f"{contracts['ommToken']}/{contracts['usds']}".encode("UTF-8"))
contracts['OMM/IUSDC'] = Address.from_data(AddressPrefix.CONTRACT,
                                           f"{contracts['ommToken']}/{contracts['iusdc']}".encode("UTF-8"))

## Deposit fee to Lending Pool

In [ ]:
call_transaction = DepositTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts["lendingPool"]) \
    .nid(3) \
    .value(DEPOSIT_ICX_AMOUNT * EXA) \
    .step_limit(1000000000) \
    .nonce(100) \
    .action("add") \
    .build()

signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx = get_tx_result(tx_hash)
response['deposit_fee_for_sharing'] = tx['status']
tx

In [ ]:
#create dummy contract address for liquidity pools
contracts['OMM/sICX'] = Address.from_data(AddressPrefix.CONTRACT,
                                          f"{contracts['ommToken']}/{contracts['sicx']}".encode("UTF-8"))
contracts['OMM/USDS'] = Address.from_data(AddressPrefix.CONTRACT,
                                          f"{contracts['ommToken']}/{contracts['usds']}".encode("UTF-8"))
contracts['OMM/IUSDC'] = Address.from_data(AddressPrefix.CONTRACT,
                                           f"{contracts['ommToken']}/{contracts['iusdc']}".encode("UTF-8"))

## Saving the pkl file


In [ ]:
now = datetime.utcnow()
filename = "new" + f'_contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'

print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)

response["pkl"] = filename

# Configuring SCOREs's parameters

In [ ]:
contract_details = [
    {'name': 'addressProvider', 'address': contracts['addressProvider']},
    {'name': 'daoFund', 'address': contracts['daoFund']},
    {'name': 'delegation', 'address': contracts['delegation']},
    {'name': 'feeProvider', 'address': contracts['feeProvider']},
    {'name': 'governance', 'address': contracts['governance']},
    {'name': 'lendingPool', 'address': contracts['lendingPool']},
    {'name': 'lendingPoolCore', 'address': contracts['lendingPoolCore']},
    {'name': 'lendingPoolDataProvider', 'address': contracts['lendingPoolDataProvider']},
    {'name': 'liquidationManager', 'address': contracts['liquidationManager']},
    {'name': 'ommToken', 'address': contracts['ommToken']},
    {'name': 'priceOracle', 'address': contracts['priceOracle']},
    {'name': 'bandOracle', 'address': contracts['bandOracle']},
    {'name': 'bridgeOToken', 'address': contracts['oUSDS']},
    {'name': 'rewards', 'address': contracts['rewardDistribution']},
    {'name': 'workerToken', 'address': contracts['workerToken']},
    {'name': 'sICX', 'address': contracts['sicx']},
    {'name': 'USDS', 'address': contracts['usds']},
    {'name': 'IUSDC', 'address': contracts['iusdc']},
    {'name': 'staking', 'address': contracts['staking']},
    {'name': 'oUSDS', 'address': contracts['oUSDS']},
    {'name': 'dUSDS', 'address': contracts['dUSDS']},
    {'name': 'oICX', 'address': contracts['oICX']},
    {'name': 'dICX', 'address': contracts['dICX']},
    {'name': 'oIUSDC', 'address': contracts['oIUSDC']},
    {'name': 'dIUSDC', 'address': contracts['dIUSDC']},
    {'name': 'stakedLP', 'address': contracts['stakedLp']},
    {'name': 'dex', 'address': contracts['dex']}
]

setting_addressProvider = [
    {'contract': 'addressProvider', 'method': 'setAddresses', 'params': {'_addressDetails': contract_details}},
    {'contract': 'addressProvider', 'method': 'setSCOREAddresses', 'params': {}},
]

for sett in setting_addressProvider:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder() \
        .from_(deployer_wallet.get_address()) \
        .to(contracts[sett['contract']]) \
        .value(0) \
        .step_limit(10000000) \
        .nid(NID) \
        .nonce(100) \
        .method(sett['method']) \
        .params(sett['params']) \
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    print(tx_hash)
    print(get_tx_result(tx_hash))
    status = get_tx_result(tx_hash)['status']
    response[f"{sett['method']} in {sett['contract']}"] = status


In [ ]:
assert response['setAddresses in addressProvider'] == 1
assert response['setSCOREAddresses in addressProvider'] == 1
print("ADDRESS PROVIDER CONFIGURED CORRECTLY")

In [ ]:
settings = [
    {'key': 'setFeeSharingTxnLimit', 'contract': 'lendingPool', 'method': 'setFeeSharingTxnLimit',
     'params': {'_limit': FEE_SHARING_TX_LIMIT}},
    {'key': 'setLoanOriginationFeePercentage', 'contract': 'feeProvider', 'method': 'setLoanOriginationFeePercentage',
     'params': {'_percentage': LOAN_ORIGINATION_PERCENTAGE}},
    {'key': 'setSymbol (usds)', 'contract': 'lendingPoolDataProvider', 'method': 'setSymbol',
     'params': {'_reserve': contracts['usds'], '_sym': "USDS"}},
    {'key': 'setSymbol (sicx)', 'contract': 'lendingPoolDataProvider', 'method': 'setSymbol',
     'params': {'_reserve': contracts['sicx'], '_sym': "ICX"}},
    {'key': 'setSymbol (iusdc)', 'contract': 'lendingPoolDataProvider', 'method': 'setSymbol',
     'params': {'_reserve': contracts['iusdc'], '_sym': "USDC"}},
    {'key': 'set_reference_data (usds)', 'contract': 'priceOracle', 'method': 'set_reference_data',
     'params': {'_base': 'USDS', '_quote': 'USD', '_rate': 1 * 10 ** 18}},
    {'key': 'set_reference_data (icx)', 'contract': 'priceOracle', 'method': 'set_reference_data',
     'params': {'_base': 'ICX', '_quote': 'USD', '_rate': 10 * 10 ** 17}},
    {'key': 'set_reference_data (iusdc)', 'contract': 'priceOracle', 'method': 'set_reference_data',
     'params': {'_base': 'IUSDC', '_quote': 'USD', '_rate': 10 * 10 ** 17}},
    {'key': 'addAllContributors', 'contract': 'delegation', 'method': 'addAllContributors',
     'params': {'_preps': PREP_LIST}},
    {'key': 'setMinimumStake', 'contract': 'ommToken', 'method': 'setMinimumStake',
     'params': {'_min': f'{MINIMUM_OMM_STAKE}'}}
]

for sett in settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder() \
        .from_(deployer_wallet.get_address()) \
        .to(contracts[sett['contract']]) \
        .value(0) \
        .step_limit(10000000) \
        .nid(NID) \
        .nonce(100) \
        .method(sett['method']) \
        .params(sett['params']) \
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_result = get_tx_result(tx_hash)
    status = tx_result['status']
    print(status)
    if status == 0:
        print(tx_result)
    response[f"{sett['key']} in {sett['contract']}"] = status

In [ ]:
asset_configs = [
    {
        'contract': 'governance', 'method': 'addPools',
        'params': {
            "_assetConfig":
                [
                    # reserves
                    {"poolID": f'{-1}', "rewardEntity": "lendingBorrow", "asset": contracts["oUSDS"],
                     "assetName": "oUSDS",
                     "distPercentage": f"{OUSDS_EMISSION}"},
                    {"poolID": f'{-1}', "rewardEntity": "lendingBorrow", "asset": contracts["dUSDS"],
                     "assetName": "dUSDS",
                     "distPercentage": f"{DUSDS_EMISSION}"},
                    {"poolID": f'{-1}', "rewardEntity": "lendingBorrow", "asset": contracts["dICX"],
                     "assetName": "dICX",
                     "distPercentage": f"{DICX_EMISSION}"},
                    {"poolID": f'{-1}', "rewardEntity": "lendingBorrow", "asset": contracts["oICX"],
                     "assetName": "oICX",
                     "distPercentage": f"{OICX_EMISSION}"},
                    {"poolID": f'{-1}', "rewardEntity": "lendingBorrow", "asset": contracts["oIUSDC"],
                     "assetName": "oIUSDC", "distPercentage": f"{OIUSDC_EMISSION}"},
                    {"poolID": f'{-1}', "rewardEntity": "lendingBorrow", "asset": contracts["dIUSDC"],
                     "assetName": "dIUSDC", "distPercentage": f"{DIUSDC_EMISSION}"},
                    # liquidity providers
                    {"poolID": f'{OMM_SICX_ID}', "rewardEntity": "liquidityProvider", "asset": contracts["OMM/sICX"],
                     "assetName": "OMM/sICX", "distPercentage": f"{OMM_SICX_DIST_PERCENTAGE}"},
                    {"poolID": f'{OMM_USDS_ID}', "rewardEntity": "liquidityProvider", "asset": contracts["OMM/USDS"],
                     "assetName": "OMM/USDS", "distPercentage": f"{OMM_USDS_DIST_PERCENTAGE}"},
                    {"poolID": f'{OMM_USDC_ID}', "rewardEntity": "liquidityProvider", "asset": contracts["OMM/IUSDC"],
                     "assetName": "OMM/IUSDC", "distPercentage": f"{OMM_USDC_DIST_PERCENTAGE}"},
                    # omm token
                    {"poolID": f'{-1}', "rewardEntity": "liquidityProvider", "asset": contracts["ommToken"],
                     "assetName": "OMM", "distPercentage": f"{OMM_DIST_PERCENTAGE}"}
                ]
        }
    }
]

for sett in asset_configs:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder() \
        .from_(deployer_wallet.get_address()) \
        .to(contracts[sett['contract']]) \
        .value(0) \
        .step_limit(10000000) \
        .nid(NID) \
        .nonce(100) \
        .method(sett['method']) \
        .params(sett['params']) \
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    status = get_tx_result(tx_hash)['status']
    print(status)
    response[f"{sett['method']} in {sett['contract']}"] = status

In [ ]:
# was in lending pool core, not in governance
settings_reserves = [
    {'contract': 'governance',
     'key': 'setReserveConstants (usds)',
     'method': 'setReserveConstants',
     'params': {"_constants": [{"reserve": contracts['usds'],
                                "optimalUtilizationRate": f"8{'0' * 17}",
                                "baseBorrowRate": f"2{'0' * 16}",
                                "slopeRate1": f"6{'0' * 16}",
                                "slopeRate2": f"1{'0' * 18}"}]}},
    {'contract': 'governance',
     'key': 'setReserveConstants (sicx)',
     'method': 'setReserveConstants',
     'params': {"_constants": [{"reserve": contracts['sicx'],
                                "optimalUtilizationRate": f"8{'0' * 17}",
                                "baseBorrowRate": f"0{'0' * 17}",
                                "slopeRate1": f"8{'0' * 16}",
                                "slopeRate2": f"2{'0' * 18}"}]}},
    {'contract': 'governance',
     'key': 'setReserveConstants (iusdc)',
     'method': 'setReserveConstants',
     'params': {"_constants": [{"reserve": contracts['iusdc'],
                                "optimalUtilizationRate": f"8{'0' * 17}",
                                "baseBorrowRate": f"2{'0' * 16}",
                                "slopeRate1": f"6{'0' * 16}",
                                "slopeRate2": f"1{'0' * 18}"}]}}
]

for sett in settings_reserves:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder() \
        .from_(deployer_wallet.get_address()) \
        .to(contracts[sett['contract']]) \
        .value(0) \
        .step_limit(10000000) \
        .nid(NID) \
        .nonce(100) \
        .method(sett['method']) \
        .params(sett['params']) \
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    status = get_tx_result(tx_hash)['status']
    print(status)
    response[f"{sett['key']} in {sett['contract']}"] = status

## Adding USDS reserve in LendingPoolCore via Governance



In [ ]:
# initially was in lending pool core, now governance controls it
params = {
    "_reserve": {
        "reserveAddress": contracts['usds'],
        "oTokenAddress": contracts['oUSDS'],
        "dTokenAddress": contracts['dUSDS'],
        "lastUpdateTimestamp": "0",
        "liquidityRate": "0",
        "borrowRate": "0",
        "liquidityCumulativeIndex": f"1{'0' * 18}",
        "borrowCumulativeIndex": f"1{'0' * 18}",
        "baseLTVasCollateral": "500000000000000000",
        "liquidationThreshold": "650000000000000000",
        "liquidationBonus": "100000000000000000",
        "decimals": "18",
        "borrowingEnabled": "1",
        "usageAsCollateralEnabled": "1",
        "isFreezed": "0",
        "isActive": "1"
    }
}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts['governance']) \
    .nid(NID) \
    .nonce(100) \
    .method("initializeReserve") \
    .params(params) \
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet, step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

status = get_tx_result(tx_hash)['status']
print(status)
response['add_usds_reserve'] = status

## Adding ICX reserve

In [ ]:
# initially was in lending pool core, now governance controls it
params = {
    "_reserve": {
        "reserveAddress": contracts['sicx'],
        "oTokenAddress": contracts['oICX'],
        "dTokenAddress": contracts['dICX'],
        "lastUpdateTimestamp": "0",
        "liquidityRate": "0",
        "borrowRate": "0",
        "liquidityCumulativeIndex": f"1{'0' * 18}",
        "borrowCumulativeIndex": f"1{'0' * 18}",
        "baseLTVasCollateral": "500000000000000000",
        "liquidationThreshold": "650000000000000000",
        "liquidationBonus": "100000000000000000",
        "decimals": "18",
        "borrowingEnabled": "1",
        "usageAsCollateralEnabled": "1",
        "isFreezed": "0",
        "isActive": "1"
    }
}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts['governance']) \
    .nid(NID) \
    .nonce(100) \
    .method("initializeReserve") \
    .params(params) \
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet, step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

status = get_tx_result(tx_hash)['status']
print(status)

response['add_icx_reserve'] = status

## Adding IUSDC reserve

In [ ]:
# initially was in lending pool core, now governance controls it
params = {
    "_reserve": {
        "reserveAddress": contracts['iusdc'],
        "oTokenAddress": contracts['oIUSDC'],
        "dTokenAddress": contracts['dIUSDC'],
        "lastUpdateTimestamp": "0",
        "liquidityRate": "0",
        "borrowRate": "0",
        "liquidityCumulativeIndex": f"1{'0' * 18}",
        "borrowCumulativeIndex": f"1{'0' * 18}",
        "baseLTVasCollateral": "500000000000000000",
        "liquidationThreshold": "650000000000000000",
        "liquidationBonus": "100000000000000000",
        "decimals": "6",
        "borrowingEnabled": "1",
        "usageAsCollateralEnabled": "1",
        "isFreezed": "0",
        "isActive": "1"
    }
}

call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts['governance']) \
    .nid(NID) \
    .nonce(100) \
    .method("initializeReserve") \
    .params(params) \
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet, step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

status = get_tx_result(tx_hash)['status']
print(status)
response['add_iusdc_reserve'] = status

In [ ]:
assert response['deploy_contract_addressProvider'] == 1
assert response['deploy_contract_daoFund'] == 1
assert response['deploy_contract_delegation'] == 1
assert response['deploy_contract_feeProvider'] == 1
assert response['deploy_contract_governance'] == 1
assert response['deploy_contract_lendingPool'] == 1
assert response['deploy_contract_lendingPoolCore'] == 1
assert response['deploy_contract_lendingPoolDataProvider'] == 1
assert response['deploy_contract_liquidationManager'] == 1
assert response['deploy_contract_ommToken'] == 1
assert response['deploy_contract_priceOracle'] == 1
assert response['deploy_contract_rewardDistribution'] == 1
assert response['deploy_contract_workerToken'] == 1
assert response['deploy_contract_stakedLp'] == 1
assert response['deploy_dToken_contract_dUSDS'] == 1
assert response['deploy_dToken_contract_dICX'] == 1
assert response['deploy_dToken_contract_dIUSDC'] == 1
assert response['deploy_oToken_contract_oUSDS'] == 1
assert response['deploy_oToken_contract_oICX'] == 1
assert response['deploy_oToken_contract_oIUSDC'] == 1
assert response['deposit_fee_for_sharing'] == 1
assert response['setFeeSharingTxnLimit in lendingPool'] == 1
assert response['setLoanOriginationFeePercentage in feeProvider'] == 1
assert response['setSymbol (usds) in lendingPoolDataProvider'] == 1
assert response['setSymbol (sicx) in lendingPoolDataProvider'] == 1
assert response['setSymbol (iusdc) in lendingPoolDataProvider'] == 1
assert response['set_reference_data (usds) in priceOracle'] == 1
assert response['set_reference_data (icx) in priceOracle'] == 1
assert response['set_reference_data (iusdc) in priceOracle'] == 1
assert response['addAllContributors in delegation'] == 1
assert response['setStartTimestamp in governance'] == 1
assert response['setMinimumStake in ommToken'] == 1
assert response['addPools in governance'] == 1
assert response['setReserveConstants (usds) in governance'] == 1
assert response['setReserveConstants (sicx) in governance'] == 1
assert response['setReserveConstants (iusdc) in governance'] == 1
assert response['add_iusdc_reserve'] == 1
assert response['add_usds_reserve'] == 1
assert response['add_icx_reserve'] == 1
print("----Configured successfully----")

## Testing deposit

In [ ]:
params = {"_amount": 5 * 10 ** 18}
call_transaction = CallTransactionBuilder()
.from_(deployer_wallet.get_address())
.to(contracts['lendingPool'])
.nid(3)
.step_limit(1000000000)
.nonce(100)
.value(5 * 10 ** 18)
.method("deposit")
.params(params)
.build()

signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)


## Testing borrow

In [ ]:
params = {"_reserve": contracts['sicx'], "_amount": 1 * 10 ** 18}

call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts['lendingPool']) \
    .nid(3) \
    .step_limit(1000000000) \
    .nonce(100) \
    .method("borrow") \
    .params(params) \
    .build()

signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)


## Updating Contracts

In [ ]:
# Updating mainnet contracts
contract_ = 'rewardDistribution'
if contract_ == 'oICX':
    update = contracts['oICX']
    contract_ = 'oToken'
elif contract_ == 'oUSDS':
    update = contracts['oUSDS']
    contract_ = 'oToken'
elif contract_ == 'oIUSDC':
    update = contracts['oIUSDC']
    contract_ = 'oToken'
elif contract_ == 'dICX':
    update = contracts['dICX']
    contract_ = 'dToken'
elif contract_ == 'dUSDS':
    update = contracts['dUSDS']
    contract_ = 'dToken'
elif contract_ == 'dIUSDC':
    update = contracts['dIUSDC']
    contract_ = 'dToken'
else:
    update = contracts[contract_]
# update = contracts[contract_]
print('update', contract_, update)

deploy_transaction = DeployTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(update) \
    .nid(NID) \
    .nonce(100) \
    .content_type("application/zip") \
    .content(gen_deploy_data_content(contract_)) \
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)
